In [25]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import time
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import os.path
from PIL import Image, ImageOps
import chardet
import re
import random
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision.transforms as transforms
import os, sys 





In [26]:
#only uncomment to load dataset locally to the vm

#os.system('gsutil -m cp -r gs://marine-clarity-307511-aiplatform / .')

# Load dataset

In [27]:
IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP',
]


def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

def dataloader(filepath):

    classes = [d for d in os.listdir(filepath) if os.path.isdir(os.path.join(filepath, d))]
    image = [img for img in classes if img.find('frames_cleanpass') > -1]
    disp  = [dsp for dsp in classes if dsp.find('disparity') > -1]

    monkaa_path = filepath + [x for x in image if 'monkaa' in x][0]
    monkaa_disp = filepath + [x for x in disp if 'monkaa' in x][0]


    monkaa_dir  = os.listdir(monkaa_path)

    all_left_img=[]
    all_right_img=[]
    all_left_disp = []
    test_left_img=[]
    test_right_img=[]
    test_left_disp = []


    for dd in monkaa_dir:
        for im in os.listdir(monkaa_path+'/'+dd+'/left/'):
            if is_image_file(monkaa_path+'/'+dd+'/left/'+im):
                all_left_img.append(monkaa_path+'/'+dd+'/left/'+im)
            all_left_disp.append(monkaa_disp+'/'+dd+'/left/'+im.split(".")[0]+'.pfm')

        for im in os.listdir(monkaa_path+'/'+dd+'/right/'):
            if is_image_file(monkaa_path+'/'+dd+'/right/'+im):
                all_right_img.append(monkaa_path+'/'+dd+'/right/'+im)
                
    print(len(all_left_img), len(all_right_img), "len monkaa")

    flying_path = filepath + [x for x in image if x == 'frames_cleanpass'][0]
    flying_disp = filepath + [x for x in disp if x == 'frames_disparity'][0]
    flying_dir = flying_path+'/TRAIN/'
    subdir = ['A','B','C']

    for ss in subdir:
        flying = os.listdir(flying_dir+ss)

        for ff in flying:

            if ff != '0201' and len(ff)==4:
                print(ff)
                imm_l = os.listdir(flying_dir+ss+'/'+ff+'/left/')
                for im in imm_l:
                    if is_image_file(flying_dir+ss+'/'+ff+'/left/'+im):
                        all_left_img.append(flying_dir+ss+'/'+ff+'/left/'+im)

                    all_left_disp.append(flying_disp+'/TRAIN/'+ss+'/'+ff+'/left/'+im.split(".")[0]+'.pfm')

                    if is_image_file(flying_dir+ss+'/'+ff+'/right/'+im):
                        all_right_img.append(flying_dir+ss+'/'+ff+'/right/'+im)
            else:
                print("0201 gone")

    flying_dir = flying_path+'/TEST/'

    subdir = ['A','B','C']

    for ss in subdir:
        flying = os.listdir(flying_dir+ss)

        for ff in flying:
            imm_l = os.listdir(flying_dir+ss+'/'+ff+'/left/')
            for im in imm_l:
                if is_image_file(flying_dir+ss+'/'+ff+'/left/'+im):
                    test_left_img.append(flying_dir+ss+'/'+ff+'/left/'+im)
          
                test_left_disp.append(flying_disp+'/TEST/'+ss+'/'+ff+'/left/'+im.split(".")[0]+'.pfm')

                if is_image_file(flying_dir+ss+'/'+ff+'/right/'+im):
                    test_right_img.append(flying_dir+ss+'/'+ff+'/right/'+im)

    print(len(all_left_img), len(all_right_img), "len 3d")

    driving_dir = filepath + [x for x in image if 'driving' in x][0] + '/'
    driving_disp = filepath + [x for x in disp if 'driving' in x][0]

    subdir1 = ['35mm_focallength','15mm_focallength']
    subdir2 = ['scene_backwards','scene_forwards']
    subdir3 = ['fast','slow']

    for i in subdir1:
        for j in subdir2:
            for k in subdir3:
                imm_l = os.listdir(driving_dir+i+'/'+j+'/'+k+'/left/')    
                for im in imm_l:
                    if is_image_file(driving_dir+i+'/'+j+'/'+k+'/left/'+im):
                        all_left_img.append(driving_dir+i+'/'+j+'/'+k+'/left/'+im)

                    all_left_disp.append(driving_disp+'/'+i+'/'+j+'/'+k+'/left/'+im.split(".")[0]+'.pfm')

                    if is_image_file(driving_dir+i+'/'+j+'/'+k+'/right/'+im):
                        all_right_img.append(driving_dir+i+'/'+j+'/'+k+'/right/'+im)
    print(len(all_left_img), len(all_right_img), "len driving")

    return all_left_img, all_right_img, all_left_disp, test_left_img, test_right_img, test_left_disp

__imagenet_stats = {'mean': [0.485, 0.456, 0.406],
                   'std': [0.229, 0.224, 0.225]}

#__imagenet_stats = {'mean': [0.5, 0.5, 0.5],
#                   'std': [0.5, 0.5, 0.5]}

__imagenet_pca = {
    'eigval': torch.Tensor([0.2175, 0.0188, 0.0045]),
    'eigvec': torch.Tensor([
        [-0.5675,  0.7192,  0.4009],
        [-0.5808, -0.0045, -0.8140],
        [-0.5836, -0.6948,  0.4203],
    ])
}


def scale_crop(input_size, scale_size=None, normalize=__imagenet_stats):
    t_list = [
        transforms.ToTensor(),
        transforms.Normalize(**normalize),
    ]
    #if scale_size != input_size:
    #t_list = [transforms.Scale((960,540))] + t_list

    return transforms.Compose(t_list)


def scale_random_crop(input_size, scale_size=None, normalize=__imagenet_stats):
    t_list = [
        transforms.RandomCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize(**normalize),
    ]
    if scale_size != input_size:
        t_list = [transforms.Scale(scale_size)] + t_list

    transforms.Compose(t_list)


def pad_random_crop(input_size, scale_size=None, normalize=__imagenet_stats):
    padding = int((scale_size - input_size) / 2)
    return transforms.Compose([
        transforms.RandomCrop(input_size, padding=padding),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(**normalize),
    ])


def inception_preproccess(input_size, normalize=__imagenet_stats):
    return transforms.Compose([
        transforms.RandomSizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(**normalize)
    ])
def inception_color_preproccess(input_size, normalize=__imagenet_stats):
    return transforms.Compose([
        #transforms.RandomSizedCrop(input_size),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        ColorJitter(
            brightness=0.4,
            contrast=0.4,
            saturation=0.4,
        ),
        Lighting(0.1, __imagenet_pca['eigval'], __imagenet_pca['eigvec']),
        transforms.Normalize(**normalize)
    ])


def get_transform(name='imagenet', input_size=None,
                  scale_size=None, normalize=None, augment=True):
    normalize = __imagenet_stats
    input_size = 256
    if augment:
            return inception_color_preproccess(input_size, normalize=normalize)
    else:
            return scale_crop(input_size=input_size,
                              scale_size=scale_size, normalize=normalize)




class Lighting(object):
    """Lighting noise(AlexNet - style PCA - based noise)"""

    def __init__(self, alphastd, eigval, eigvec):
        self.alphastd = alphastd
        self.eigval = eigval
        self.eigvec = eigvec

    def __call__(self, img):
        if self.alphastd == 0:
            return img

        alpha = img.new().resize_(3).normal_(0, self.alphastd)
        rgb = self.eigvec.type_as(img).clone()\
            .mul(alpha.view(1, 3).expand(3, 3))\
            .mul(self.eigval.view(1, 3).expand(3, 3))\
            .sum(1).squeeze()

        return img.add(rgb.view(3, 1, 1).expand_as(img))


class Grayscale(object):

    def __call__(self, img):
        gs = img.clone()
        gs[0].mul_(0.299).add_(0.587, gs[1]).add_(0.114, gs[2])
        gs[1].copy_(gs[0])
        gs[2].copy_(gs[0])
        return gs


class Saturation(object):

    def __init__(self, var):
        self.var = var

    def __call__(self, img):
        gs = Grayscale()(img)
        alpha = random.uniform(0, self.var)
        return img.lerp(gs, alpha)


class Brightness(object):

    def __init__(self, var):
        self.var = var

    def __call__(self, img):
        gs = img.new().resize_as_(img).zero_()
        alpha = random.uniform(0, self.var)
        return img.lerp(gs, alpha)


class Contrast(object):

    def __init__(self, var):
        self.var = var

    def __call__(self, img):
        gs = Grayscale()(img)
        gs.fill_(gs.mean())
        alpha = random.uniform(0, self.var)
        return img.lerp(gs, alpha)


class RandomOrder(object):
    """ Composes several transforms together in random order.
    """

    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img):
        if self.transforms is None:
            return img
        order = torch.randperm(len(self.transforms))
        for i in order:
            img = self.transforms[i](img)
        return img


class ColorJitter(RandomOrder):

    def __init__(self, brightness=0.4, contrast=0.4, saturation=0.4):
        self.transforms = []
        if brightness != 0:
            self.transforms.append(Brightness(brightness))
        if contrast != 0:
            self.transforms.append(Contrast(contrast))
        if saturation != 0:
            self.transforms.append(Saturation(saturation))

def readPFM(file):
    file = open(file, 'rb')

    color = None
    width = None
    height = None
    scale = None
    endian = None

    header = file.readline().rstrip()
    encode_type = chardet.detect(header)  
    header = header.decode(encode_type['encoding'])
    if header == 'PF':
        color = True
    elif header == 'Pf':
        color = False
    else:
        raise Exception('Not a PFM file.')

    dim_match = re.match(r'^(\d+)\s(\d+)\s$', file.readline().decode(encode_type['encoding']))
    if dim_match:
        width, height = map(int, dim_match.groups())
    else:
        raise Exception('Malformed PFM header.')

    scale = float(file.readline().rstrip().decode(encode_type['encoding']))
    if scale < 0: # little-endian
        endian = '<'
        scale = -scale
    else:
        endian = '>' # big-endian
    
    data = np.fromfile(file, endian + 'f')
    shape = (height, width, 3) if color else (height, width)

    try:
        data = np.reshape(data, shape)
    except:
        print(file)
        print("not the right shape, use only zeros")
        data = np.zeros((540, 960))
    
    data = np.flipud(data)
    return data, scale


import torch.utils.data as data

IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP',
]

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

def default_loader(path):
    return Image.open(path).convert('RGB')

def disparity_loader(path):
    return readPFM(path)

class myImageFloder(data.Dataset):
    def __init__(self, left, right, left_disparity, training, loader=default_loader, dploader= disparity_loader):
 
        self.left = left
        self.right = right
        self.disp_L = left_disparity
        self.loader = loader
        self.dploader = dploader
        self.training = training

    def __getitem__(self, index):
        left  = self.left[index]
        right = self.right[index]
        disp_L= self.disp_L[index]


        left_img = self.loader(left)
        right_img = self.loader(right)
        dataL, scaleL = self.dploader(disp_L)
        dataL = np.ascontiguousarray(dataL,dtype=np.float32)

        if self.training:  
            w, h = left_img.size
            th, tw = 256, 512

            x1 = random.randint(0, w - tw)
            y1 = random.randint(0, h - th)

            left_img = left_img.crop((x1, y1, x1 + tw, y1 + th))
            right_img = right_img.crop((x1, y1, x1 + tw, y1 + th))

            dataL = dataL[y1:y1 + th, x1:x1 + tw]

            processed = get_transform(augment=False)  
            left_img   = processed(left_img)
            right_img  = processed(right_img)

            return left_img, right_img, dataL
        else:
            processed = get_transform(augment=False)  
            left_img       = processed(left_img)
            right_img      = processed(right_img) 
            return left_img, right_img, dataL

    def __len__(self):
        return len(self.left)

    
def convbn(in_planes, out_planes, kernel_size, stride, pad, dilation):

    return nn.Sequential(nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=dilation if dilation > 1 else pad, dilation = dilation, bias=False),
                         nn.BatchNorm2d(out_planes))


def convbn_3d(in_planes, out_planes, kernel_size, stride, pad):

    return nn.Sequential(nn.Conv3d(in_planes, out_planes, kernel_size=kernel_size, padding=pad, stride=stride,bias=False),
                         nn.BatchNorm3d(out_planes))



# Models

In [28]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, inplanes, planes, stride, downsample, pad, dilation):
        super(BasicBlock, self).__init__()

        self.conv1 = nn.Sequential(convbn(inplanes, planes, 3, stride, pad, dilation),
                                   nn.ReLU(inplace=True))

        self.conv2 = convbn(planes, planes, 3, 1, pad, dilation)

        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)

        if self.downsample is not None:
            x = self.downsample(x)

        out += x

        return out

class disparityregression(nn.Module):
    def __init__(self, maxdisp):
        super(disparityregression, self).__init__()
        self.disp = torch.Tensor(np.reshape(np.array(range(maxdisp)),[1, maxdisp,1,1])).cuda()

    def forward(self, x):
        out = torch.sum(x*self.disp.data,1, keepdim=True)
        return out

class feature_extraction(nn.Module):
    def __init__(self):
        super(feature_extraction, self).__init__()
        self.inplanes = 32
        self.firstconv = nn.Sequential(convbn(3, 32, 3, 2, 1, 1),
                                       nn.ReLU(inplace=True),
                                       convbn(32, 32, 3, 1, 1, 1),
                                       nn.ReLU(inplace=True),
                                       convbn(32, 32, 3, 1, 1, 1),
                                       nn.ReLU(inplace=True))

        self.layer1 = self._make_layer(BasicBlock, 32, 3, 1,1,1)
        self.layer2 = self._make_layer(BasicBlock, 64, 16, 2,1,1) 
        self.layer3 = self._make_layer(BasicBlock, 128, 3, 1,1,1)
        self.layer4 = self._make_layer(BasicBlock, 128, 3, 1,1,2)

        self.branch1 = nn.Sequential(nn.AvgPool2d((64, 64), stride=(64,64)),
                                     convbn(128, 32, 1, 1, 0, 1),
                                     nn.ReLU(inplace=True))

        self.branch2 = nn.Sequential(nn.AvgPool2d((32, 32), stride=(32,32)),
                                     convbn(128, 32, 1, 1, 0, 1),
                                     nn.ReLU(inplace=True))

        self.branch3 = nn.Sequential(nn.AvgPool2d((16, 16), stride=(16,16)),
                                     convbn(128, 32, 1, 1, 0, 1),
                                     nn.ReLU(inplace=True))

        self.branch4 = nn.Sequential(nn.AvgPool2d((8, 8), stride=(8,8)),
                                     convbn(128, 32, 1, 1, 0, 1),
                                     nn.ReLU(inplace=True))

        self.lastconv = nn.Sequential(convbn(320, 128, 3, 1, 1, 1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(128, 32, kernel_size=1, padding=0, stride = 1, bias=False))

    def _make_layer(self, block, planes, blocks, stride, pad, dilation):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
           downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),)

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, pad, dilation))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes,1,None,pad,dilation))

        return nn.Sequential(*layers)

    def forward(self, x):
        output      = self.firstconv(x)
        output      = self.layer1(output)
        output_raw  = self.layer2(output)
        output      = self.layer3(output_raw)
        output_skip = self.layer4(output)


        output_branch1 = self.branch1(output_skip)
        output_branch1 = F.interpolate(output_branch1, (output_skip.size()[2],output_skip.size()[3]),mode='bilinear', align_corners=True)

        output_branch2 = self.branch2(output_skip)
        output_branch2 = F.interpolate(output_branch2, (output_skip.size()[2],output_skip.size()[3]),mode='bilinear', align_corners=True)

        output_branch3 = self.branch3(output_skip)
        output_branch3 = F.interpolate(output_branch3, (output_skip.size()[2],output_skip.size()[3]),mode='bilinear', align_corners=True)

        output_branch4 = self.branch4(output_skip)
        output_branch4 = F.interpolate(output_branch4, (output_skip.size()[2],output_skip.size()[3]),mode='bilinear', align_corners=True)

        output_feature = torch.cat((output_raw, output_skip, output_branch4, output_branch3, output_branch2, output_branch1), 1)
        output_feature = self.lastconv(output_feature)

        return output_feature

class PSMNet_basic(nn.Module):
    def __init__(self, maxdisp):
        super(PSMNet_basic, self).__init__()
        self.maxdisp = maxdisp
        self.feature_extraction = feature_extraction()

########
        self.dres0 = nn.Sequential(convbn_3d(64, 32, 3, 1, 1),
                                     nn.ReLU(inplace=True),
                                     convbn_3d(32, 32, 3, 1, 1),
                                     nn.ReLU(inplace=True))

        self.dres1 = nn.Sequential(convbn_3d(32, 32, 3, 1, 1),
                                   nn.ReLU(inplace=True),
                                   convbn_3d(32, 32, 3, 1, 1)) 

        self.dres2 = nn.Sequential(convbn_3d(32, 32, 3, 1, 1),
                                   nn.ReLU(inplace=True),
                                   convbn_3d(32, 32, 3, 1, 1))
 
        self.dres3 = nn.Sequential(convbn_3d(32, 32, 3, 1, 1),
                                   nn.ReLU(inplace=True),
                                   convbn_3d(32, 32, 3, 1, 1)) 

        self.dres4 = nn.Sequential(convbn_3d(32, 32, 3, 1, 1),
                                   nn.ReLU(inplace=True),
                                   convbn_3d(32, 32, 3, 1, 1)) 
 
        self.classify = nn.Sequential(convbn_3d(32, 32, 3, 1, 1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv3d(32, 1, kernel_size=3, padding=1, stride=1,bias=False))


        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.Conv3d):
                n = m.kernel_size[0] * m.kernel_size[1]*m.kernel_size[2] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()


    def forward(self, left, right):

        refimg_fea     = self.feature_extraction(left)
        targetimg_fea  = self.feature_extraction(right)
 
        #matching
        cost = Variable(torch.FloatTensor(refimg_fea.size()[0], refimg_fea.size()[1]*2, self.maxdisp/4,  refimg_fea.size()[2],  refimg_fea.size()[3]).zero_(), volatile= not self.training).cuda()

        for i in range(self.maxdisp/4):
            if i > 0 :
             cost[:, :refimg_fea.size()[1], i, :,i:]   = refimg_fea[:,:,:,i:]
             cost[:, refimg_fea.size()[1]:, i, :,i:] = targetimg_fea[:,:,:,:-i]
            else:
             cost[:, :refimg_fea.size()[1], i, :,:]   = refimg_fea
             cost[:, refimg_fea.size()[1]:, i, :,:]   = targetimg_fea
        cost = cost.contiguous()

        cost0 = self.dres0(cost)
        cost0 = self.dres1(cost0) + cost0
        cost0 = self.dres2(cost0) + cost0 
        cost0 = self.dres3(cost0) + cost0 
        cost0 = self.dres4(cost0) + cost0

        cost = self.classify(cost0)
        cost = F.upsample(cost, [self.maxdisp,left.size()[2],left.size()[3]], mode='trilinear')
        cost = torch.squeeze(cost,1)
        pred = F.softmax(cost)
        pred = disparityregression(self.maxdisp)(pred)

        return pred
class hourglass(nn.Module):
    def __init__(self, inplanes):
        super(hourglass, self).__init__()

        self.conv1 = nn.Sequential(convbn_3d(inplanes, inplanes*2, kernel_size=3, stride=2, pad=1),
                                   nn.ReLU(inplace=True))

        self.conv2 = convbn_3d(inplanes*2, inplanes*2, kernel_size=3, stride=1, pad=1)

        self.conv3 = nn.Sequential(convbn_3d(inplanes*2, inplanes*2, kernel_size=3, stride=2, pad=1),
                                   nn.ReLU(inplace=True))

        self.conv4 = nn.Sequential(convbn_3d(inplanes*2, inplanes*2, kernel_size=3, stride=1, pad=1),
                                   nn.ReLU(inplace=True))

        self.conv5 = nn.Sequential(nn.ConvTranspose3d(inplanes*2, inplanes*2, kernel_size=3, padding=1, output_padding=1, stride=2,bias=False),
                                   nn.BatchNorm3d(inplanes*2)) #+conv2

        self.conv6 = nn.Sequential(nn.ConvTranspose3d(inplanes*2, inplanes, kernel_size=3, padding=1, output_padding=1, stride=2,bias=False),
                                   nn.BatchNorm3d(inplanes)) #+x

    def forward(self, x ,presqu, postsqu):
        
        out  = self.conv1(x) #in:1/4 out:1/8
        pre  = self.conv2(out) #in:1/8 out:1/8
        if postsqu is not None:
           pre = F.relu(pre + postsqu, inplace=True)
        else:
           pre = F.relu(pre, inplace=True)

        out  = self.conv3(pre) #in:1/8 out:1/16
        out  = self.conv4(out) #in:1/16 out:1/16

        if presqu is not None:
           post = F.relu(self.conv5(out)+presqu, inplace=True) #in:1/16 out:1/8
        else:
           post = F.relu(self.conv5(out)+pre, inplace=True) 

        out  = self.conv6(post)  #in:1/8 out:1/4

        return out, pre, post

class PSMNet_stackhourglass(nn.Module):
    def __init__(self, maxdisp):
        super(PSMNet_stackhourglass, self).__init__()
        self.maxdisp = maxdisp

        self.feature_extraction = feature_extraction()

        self.dres0 = nn.Sequential(convbn_3d(64, 32, 3, 1, 1),
                                     nn.ReLU(inplace=True),
                                     convbn_3d(32, 32, 3, 1, 1),
                                     nn.ReLU(inplace=True))

        self.dres1 = nn.Sequential(convbn_3d(32, 32, 3, 1, 1),
                                   nn.ReLU(inplace=True),
                                   convbn_3d(32, 32, 3, 1, 1)) 

        self.dres2 = hourglass(32)

        self.dres3 = hourglass(32)

        self.dres4 = hourglass(32)

        self.classif1 = nn.Sequential(convbn_3d(32, 32, 3, 1, 1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv3d(32, 1, kernel_size=3, padding=1, stride=1,bias=False))

        self.classif2 = nn.Sequential(convbn_3d(32, 32, 3, 1, 1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv3d(32, 1, kernel_size=3, padding=1, stride=1,bias=False))

        self.classif3 = nn.Sequential(convbn_3d(32, 32, 3, 1, 1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv3d(32, 1, kernel_size=3, padding=1, stride=1,bias=False))

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.Conv3d):
                n = m.kernel_size[0] * m.kernel_size[1]*m.kernel_size[2] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()


    def forward(self, left, right):

        refimg_fea     = self.feature_extraction(left)
        targetimg_fea  = self.feature_extraction(right)


        #matching
        cost = Variable(torch.FloatTensor(refimg_fea.size()[0], refimg_fea.size()[1]*2, self.maxdisp//4,  refimg_fea.size()[2],  refimg_fea.size()[3]).zero_()).cuda()

        for i in range(self.maxdisp//4):
            if i > 0 :
             cost[:, :refimg_fea.size()[1], i, :,i:]   = refimg_fea[:,:,:,i:]
             cost[:, refimg_fea.size()[1]:, i, :,i:] = targetimg_fea[:,:,:,:-i]
            else:
             cost[:, :refimg_fea.size()[1], i, :,:]   = refimg_fea
             cost[:, refimg_fea.size()[1]:, i, :,:]   = targetimg_fea
        cost = cost.contiguous()

        cost0 = self.dres0(cost)
        cost0 = self.dres1(cost0) + cost0

        out1, pre1, post1 = self.dres2(cost0, None, None) 
        out1 = out1+cost0

        out2, pre2, post2 = self.dres3(out1, pre1, post1) 
        out2 = out2+cost0

        out3, pre3, post3 = self.dres4(out2, pre1, post2) 
        out3 = out3+cost0

        cost1 = self.classif1(out1)
        cost2 = self.classif2(out2) + cost1
        cost3 = self.classif3(out3) + cost2

        if self.training:
            cost1 = F.interpolate(cost1, [self.maxdisp,left.size()[2],left.size()[3]], mode='trilinear', align_corners=True)
            cost2 = F.interpolate(cost2, [self.maxdisp,left.size()[2],left.size()[3]], mode='trilinear', align_corners=True)

            cost1 = torch.squeeze(cost1,1)
            pred1 = F.softmax(cost1,dim=1)
            pred1 = disparityregression(self.maxdisp)(pred1)

            cost2 = torch.squeeze(cost2,1)
            pred2 = F.softmax(cost2,dim=1)
            pred2 = disparityregression(self.maxdisp)(pred2)

        cost3 = F.interpolate(cost3, [self.maxdisp,left.size()[2],left.size()[3]], mode='trilinear', align_corners=True)
        cost3 = torch.squeeze(cost3,1)
        pred3 = F.softmax(cost3,dim=1)
        #For your information: This formulation 'softmax(c)' learned "similarity" 
        #while 'softmax(-c)' learned 'matching cost' as mentioned in the paper.
        #However, 'c' or '-c' do not affect the performance because feature-based cost volume provided flexibility.
        pred3 = disparityregression(self.maxdisp)(pred3)

        if self.training:
            return pred1, pred2, pred3
        else:
            return pred3

        

# Training and testing

In [29]:
        
# parser = argparse.ArgumentParser(description='PSMNet')
# parser.add_argument('--maxdisp', type=int, default=192,
#                     help='maxium disparity')
# parser.add_argument('--model', default='stackhourglass',
#                     help='select model')
# parser.add_argument('--datapath', default='/media/susan/Elements/dataset/',
#                     help='datapath')
# parser.add_argument('--epochs', type=int, default=10,
#                     help='number of epochs to train')
# parser.add_argument('--loadmodel', default=None,
#                     help='load model')
# parser.add_argument('--savemodel', default='./',
#                     help='save model')
# parser.add_argument('--no-cuda', action='store_true', default=False,
#                     help='enables CUDA training')
# parser.add_argument('--seed', type=int, default=1, metavar='S',
#                     help='random seed (default: 1)')
# args = parser.parse_args()
# args.cuda = not args.no_cuda and torch.cuda.is_available()
seed=2


datapath = 'marine-clarity-307511-aiplatform/dataset/'
model_name='stackhourglass'
cuda=True
loadmodel=None
maxdisp=192
savemodel='./'
epochs=10

if cuda:
    torch.cuda.manual_seed(seed)

all_left_img, all_right_img, all_left_disp, test_left_img, test_right_img, test_left_disp = dataloader(datapath)

TrainImgLoader = torch.utils.data.DataLoader(
    myImageFloder(all_left_img, all_right_img, all_left_disp, True),
    batch_size=5, shuffle=True, num_workers=0, drop_last=False)

TestImgLoader = torch.utils.data.DataLoader(
    myImageFloder(test_left_img, test_right_img, test_left_disp, False),
    batch_size=5, shuffle=False, num_workers=0, drop_last=False)



print(len(TrainImgLoader), "len train")
print(len(TestImgLoader), "len test")

if model_name == 'stackhourglass':
    model = PSMNet_stackhourglass(maxdisp)
elif model_name == 'basic':
    model = PSMNet_basic(maxdisp)
else:
    print('no model')

if cuda:
    model = nn.DataParallel(model)
    model.cuda()


    
if loadmodel is not None:
    print('Load pretrained model')
    pretrain_dict = torch.load(loadmodel)
    model.load_state_dict(pretrain_dict['state_dict'])
#     optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))
#     optimizer.load_state_dict(pretrain_dict['optimizer'])

optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

print('Number of model parameters: {}'.format(sum([p.data.nelement() for p in model.parameters()])))



dict_save = {}


def train(imgL, imgR, disp_L):
    model.train()

    if cuda:
        imgL, imgR, disp_true = imgL.cuda(), imgR.cuda(), disp_L.cuda()
    else:
        print("no cuda")
        disp_true=disp_L

    # ---------
    mask = disp_true < maxdisp
    mask.detach_()
    # ----
    optimizer.zero_grad()

    if model_name == 'stackhourglass':
        output1, output2, output3 = model(imgL, imgR)
        output1 = torch.squeeze(output1, 1)
        output2 = torch.squeeze(output2, 1)
        output3 = torch.squeeze(output3, 1)
        loss = 0.5 * F.smooth_l1_loss(output1[mask], disp_true[mask], size_average=True) + 0.7 * F.smooth_l1_loss(
            output2[mask], disp_true[mask], size_average=True) + F.smooth_l1_loss(output3[mask], disp_true[mask],
                                                                                  size_average=True)
    elif model_name == 'basic':
        output = model(imgL, imgR)
        output = torch.squeeze(output, 1)
        loss = F.smooth_l1_loss(output[mask], disp_true[mask], size_average=True)

    loss.backward()
    optimizer.step()

    return loss.data


def test(imgL, imgR, disp_true):
    # print(model)
    model.eval()

    if cuda:
        imgL, imgR, disp_true = imgL.cuda(), imgR.cuda(), disp_true.cuda()
    # ---------
    mask = disp_true < maxdisp
    mask.detach_()
    # ----

    if imgL.shape[2] % 16 != 0:
        times = imgL.shape[2] // 16
        top_pad = (times + 1) * 16 - imgL.shape[2]
    else:
        top_pad = 0

    if imgL.shape[3] % 16 != 0:
        times = imgL.shape[3] // 16
        right_pad = (times + 1) * 16 - imgL.shape[3]
    else:
        right_pad = 0

    imgL = F.pad(imgL, (0, right_pad, top_pad, 0))
    imgR = F.pad(imgR, (0, right_pad, top_pad, 0))

    with torch.no_grad():
        output3 = model(imgL, imgR)
        output3 = torch.squeeze(output3, dim=1)

    if top_pad != 0:
        img = output3[:, top_pad:, :]
    else:
        img = output3


    if len(disp_true[mask]) == 0:
        loss = 0
    else:
        loss = torch.mean(torch.abs(img[mask]-disp_true[mask]))   # F.l1_loss(img[mask],disp_true[mask]) #torch.mean(torch.abs(img[mask]-disp_true[mask]))  # end-point-error
        loss = loss.data.cuda()

    torch.cuda.empty_cache()
    return loss


def adjust_learning_rate(optimizer, epoch):
    lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def main():
    start_full_time = time.time()
    for epoch in range(0, epochs):
        print('This is %d-th epoch' % (epoch))
        total_train_loss = 0
        adjust_learning_rate(optimizer, epoch)

        ## training ##
        for batch_idx, (imgL_crop, imgR_crop, disp_crop_L) in enumerate(TrainImgLoader):
            start_time = time.time()

            loss = train(imgL_crop, imgR_crop, disp_crop_L)
            print('Iter %d training loss = %.3f , time = %.2f' %(batch_idx, loss, time.time() - start_time))
            total_train_loss += float(loss)
                
        print('epoch %d total training loss = %.3f' % (epoch, total_train_loss / len(TrainImgLoader)))

        # SAVE
        savefilename = savemodel + '/checkpoint_full' + str(epoch) + '.tar'
        torch.save({
            'epoch': epoch,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'train_loss': total_train_loss / len(TrainImgLoader),
        }, savefilename)

        dict_save[epoch] = total_train_loss

    print(dict_save)
    print('full training time = %.2f HR' % ((time.time() - start_full_time) / 3600))

    # ------------- TEST ------------------------------------------------------------
    file_test = open("test_sceneflow_cloud_trained_full.txt", "w")
    total_test_loss = 0
    for batch_idx, (imgL, imgR, disp_L) in enumerate(TestImgLoader):
        test_loss = test(imgL, imgR, disp_L)
        print('Iter %d test loss = %.3f' %(batch_idx, test_loss))
        total_test_loss += float(test_loss)
        tot_str = str(test_loss) + ', '
        file_test.write(tot_str)
    file_test.close()
    print('total test loss = %.3f' % (total_test_loss / len(TestImgLoader)))
    # ----------------------------------------------------------------------------------
    # SAVE test information
    savefilename = savemodel + 'testinformation_full.tar'
    torch.save({
        'test_loss': total_test_loss / len(TestImgLoader),
    }, savefilename)


if __name__ == '__main__':
    main()


8664 8664 len monkaa
0099
0743
0129
0507
0694
0567
0667
0290
0506
0149
0053
0055
0046
0337
0455
0460
0467
0704
0424
0729
0427
0063
0291
0335
0322
0166
0643
0477
0072
0230
0581
0075
0240
0636
0233
0350
0550
0721
0566
0658
0058
0188
0332
0271
0511
0108
0615
0473
0321
0597
0146
0394
0680
0330
0547
0118
0387
0429
0418
0518
0232
0629
0202
0327
0416
0522
0417
0381
0343
0714
0347
0304
0295
0045
0031
0148
0036
0062
0193
0521
0401
0336
0093
0086
0480
0163
0486
0602
0705
0326
0447
0552
0293
0390
0673
0621
0476
0159
0463
0668
0324
0228
0022
0373
0465
0445
0377
0652
0726
0266
0505
0223
0733
0137
0084
0116
0214
0164
0065
0154
0641
0540
0328
0655
0210
0066
0560
0526
0616
0090
0609
0691
0490
0748
0441
0367
0329
0439
0213
0286
0042
0542
0594
0528
0659
0585
0190
0080
0520
0375
0685
0237
0437
0555
0713
0130
0596
0008
0600
0677
0598
0323
0307
0599
0678
0413
0723
0064
0218
0496
0250
0648
0534
0737
0131
0746
0389
0692
0140
0171
0013
0358
0356
0404
0502
0181
0119
0183
0122
0094
0348
0563
0040
0313
0402
0739

0357
0001
0549
0069
0096
0355
0741
0517
0587
0736
0192
0362
0071
0151
0349
0308
0571
0709
0551
0450
0258
0559
0494
0670
0693
0582
0236
0194
0015
0156
0491
0459
0719
0669
0143
0603
0113
0410
0173
0101
0138
0640
0448
0558
0508
0479
0608
0305
0682
0665
0379
0048
0647
0478
0501
0227
0325
0420
0619
0315
0341
0318
0421
0576
0365
0431
0388
0208
0523
0225
0311
0405
0124
0451
0107
0376
0352
0003
0310
0426
0272
0569
0288
0353
0380
0257
0203
0690
0297
0433
0103
0246
0449
0612
0215
0456
0144
0200
0611
0110
0067
0395
0041
0369
0299
0592
0147
0516
0442
0535
0097
0139
0320
0443
0339
0226
0672
0675
0624
0430
0182
0027
0663
0564
0637
0623
0524
0572
0556
0338
0712
0695
0136
0472
0092
0287
0434
0646
0545
0708
0374
0503
0573
0710
0633
0495
0300
0613
0312
0504
0278
0161
0345
0009
0533
0158
0471
0179
0274
0273
0408
0720
0382
0698
0593
0391
0749
0630
0019
0393
0574
0579
0043
0725
0334
0731
0706
0530
0660
0697
0683
0180
0054
0259
0531
0049
0446
0292
0241
0727
0738
0400
0234
0744
0068
0170
0219
0037
0229
0453


Iter 125 training loss = 25.680 , time = 2.25
Iter 126 training loss = 27.039 , time = 2.25
Iter 127 training loss = 35.442 , time = 2.25
Iter 128 training loss = 25.659 , time = 2.25
Iter 129 training loss = 22.064 , time = 2.26
Iter 130 training loss = 21.869 , time = 2.25
Iter 131 training loss = 26.164 , time = 2.26
Iter 132 training loss = 22.431 , time = 2.25
Iter 133 training loss = 20.682 , time = 2.26
Iter 134 training loss = 20.160 , time = 2.25
Iter 135 training loss = 19.966 , time = 2.26
Iter 136 training loss = 24.940 , time = 2.25
Iter 137 training loss = 25.551 , time = 2.25
Iter 138 training loss = 21.480 , time = 2.26
Iter 139 training loss = 17.041 , time = 2.25
Iter 140 training loss = 27.637 , time = 2.25
Iter 141 training loss = 22.865 , time = 2.26
Iter 142 training loss = 44.420 , time = 2.25
Iter 143 training loss = 19.005 , time = 2.27
Iter 144 training loss = 17.318 , time = 2.27
Iter 145 training loss = 8.355 , time = 2.25
Iter 146 training loss = 18.496 , t

Iter 304 training loss = 14.765 , time = 2.26
Iter 305 training loss = 28.139 , time = 2.25
Iter 306 training loss = 12.516 , time = 2.24
Iter 307 training loss = 11.678 , time = 2.25
Iter 308 training loss = 21.606 , time = 2.26
Iter 309 training loss = 19.865 , time = 2.25
Iter 310 training loss = 18.741 , time = 2.27
Iter 311 training loss = 15.061 , time = 2.24
Iter 312 training loss = 15.956 , time = 2.25
Iter 313 training loss = 17.141 , time = 2.24
Iter 314 training loss = 19.005 , time = 2.26
Iter 315 training loss = 8.001 , time = 2.25
Iter 316 training loss = 26.105 , time = 2.25
Iter 317 training loss = 12.538 , time = 2.26
Iter 318 training loss = 21.389 , time = 2.26
Iter 319 training loss = 29.406 , time = 2.25
Iter 320 training loss = 36.528 , time = 2.25
Iter 321 training loss = 30.127 , time = 2.25
Iter 322 training loss = 36.032 , time = 2.26
Iter 323 training loss = 19.112 , time = 2.25
Iter 324 training loss = 12.787 , time = 2.25
Iter 325 training loss = 11.904 , t

Iter 483 training loss = 28.391 , time = 2.25
Iter 484 training loss = 12.348 , time = 2.26
Iter 485 training loss = 11.864 , time = 2.25
Iter 486 training loss = 19.515 , time = 2.26
Iter 487 training loss = 16.612 , time = 2.25
Iter 488 training loss = 11.300 , time = 2.25
Iter 489 training loss = 11.857 , time = 2.26
Iter 490 training loss = 16.138 , time = 2.25
Iter 491 training loss = 12.365 , time = 2.25
Iter 492 training loss = 15.537 , time = 2.25
Iter 493 training loss = 8.252 , time = 2.26
Iter 494 training loss = 12.558 , time = 2.26
Iter 495 training loss = 9.092 , time = 2.25
Iter 496 training loss = 15.944 , time = 2.25
Iter 497 training loss = 9.045 , time = 2.26
Iter 498 training loss = 13.723 , time = 2.26
Iter 499 training loss = 19.789 , time = 2.26
Iter 500 training loss = 14.142 , time = 2.25
Iter 501 training loss = 16.868 , time = 2.26
Iter 502 training loss = 24.209 , time = 2.26
Iter 503 training loss = 14.002 , time = 2.26
Iter 504 training loss = 16.954 , tim

...

Iter 6416 training loss = 3.511 , time = 2.27
Iter 6417 training loss = 3.579 , time = 2.26
Iter 6418 training loss = 2.537 , time = 2.27
Iter 6419 training loss = 11.058 , time = 2.27
Iter 6420 training loss = 8.743 , time = 2.27
Iter 6421 training loss = 3.594 , time = 2.26
Iter 6422 training loss = 1.581 , time = 2.27
Iter 6423 training loss = 2.747 , time = 2.27
Iter 6424 training loss = 3.014 , time = 2.27
Iter 6425 training loss = 4.876 , time = 2.27
Iter 6426 training loss = 1.592 , time = 2.26
Iter 6427 training loss = 6.378 , time = 2.27
Iter 6428 training loss = 3.777 , time = 2.28
Iter 6429 training loss = 7.721 , time = 2.27
Iter 6430 training loss = 3.014 , time = 2.25
Iter 6431 training loss = 1.793 , time = 2.26
Iter 6432 training loss = 2.665 , time = 2.27
Iter 6433 training loss = 2.927 , time = 2.25
Iter 6434 training loss = 4.325 , time = 2.26
Iter 6435 training loss = 4.796 , time = 2.27
Iter 6436 training loss = 2.289 , time = 2.27
Iter 6437 training loss = 2.992 ,

Iter 6594 training loss = 3.137 , time = 2.26
Iter 6595 training loss = 3.361 , time = 2.27
Iter 6596 training loss = 2.318 , time = 2.28
Iter 6597 training loss = 4.134 , time = 2.27
Iter 6598 training loss = 1.107 , time = 2.27
Iter 6599 training loss = 2.873 , time = 2.32
Iter 6600 training loss = 3.925 , time = 2.27
Iter 6601 training loss = 2.966 , time = 2.26
Iter 6602 training loss = 1.720 , time = 2.26
Iter 6603 training loss = 3.697 , time = 2.27
Iter 6604 training loss = 3.973 , time = 2.27
Iter 6605 training loss = 4.145 , time = 2.26
Iter 6606 training loss = 4.428 , time = 2.26
Iter 6607 training loss = 3.574 , time = 2.26
Iter 6608 training loss = 4.098 , time = 2.27
Iter 6609 training loss = 1.367 , time = 2.26
Iter 6610 training loss = 6.270 , time = 2.28
Iter 6611 training loss = 6.846 , time = 2.26
Iter 6612 training loss = 4.097 , time = 2.27
Iter 6613 training loss = 3.194 , time = 2.27
Iter 6614 training loss = 3.895 , time = 2.26
Iter 6615 training loss = 6.112 , 

Iter 6773 training loss = 2.832 , time = 2.26
Iter 6774 training loss = 7.194 , time = 2.26
Iter 6775 training loss = 2.586 , time = 2.27
Iter 6776 training loss = 13.695 , time = 2.28
Iter 6777 training loss = 2.417 , time = 2.27
Iter 6778 training loss = 0.474 , time = 2.27
Iter 6779 training loss = 2.682 , time = 2.27
Iter 6780 training loss = 1.932 , time = 2.26
Iter 6781 training loss = 4.178 , time = 2.28
Iter 6782 training loss = 2.616 , time = 2.27
Iter 6783 training loss = 4.326 , time = 2.28
Iter 6784 training loss = 4.094 , time = 2.26
Iter 6785 training loss = 9.320 , time = 2.27
Iter 6786 training loss = 5.440 , time = 2.27
Iter 6787 training loss = 1.803 , time = 2.27
Iter 6788 training loss = 3.091 , time = 2.26
Iter 6789 training loss = 3.260 , time = 2.26
Iter 6790 training loss = 2.922 , time = 2.27
Iter 6791 training loss = 4.271 , time = 2.28
Iter 6792 training loss = 0.941 , time = 2.27
Iter 6793 training loss = 2.590 , time = 2.26
Iter 6794 training loss = 2.772 ,

Iter 6951 training loss = 2.730 , time = 2.27
Iter 6952 training loss = 2.059 , time = 2.26
Iter 6953 training loss = 2.663 , time = 2.27
Iter 6954 training loss = 6.223 , time = 2.27
Iter 6955 training loss = 1.892 , time = 2.26
Iter 6956 training loss = 5.262 , time = 2.26
Iter 6957 training loss = 1.312 , time = 2.27
Iter 6958 training loss = 3.963 , time = 2.27
Iter 6959 training loss = 1.924 , time = 2.26
Iter 6960 training loss = 5.236 , time = 2.27
Iter 6961 training loss = 6.668 , time = 2.28
Iter 6962 training loss = 2.555 , time = 2.27
Iter 6963 training loss = 6.683 , time = 2.26
Iter 6964 training loss = 3.158 , time = 2.27
Iter 6965 training loss = 4.529 , time = 2.26
Iter 6966 training loss = 3.392 , time = 2.27
Iter 6967 training loss = 1.911 , time = 2.27
Iter 6968 training loss = 5.581 , time = 2.26
Iter 6969 training loss = 5.128 , time = 2.27
Iter 6970 training loss = 4.654 , time = 2.26
Iter 6971 training loss = 5.382 , time = 2.28
Iter 6972 training loss = 3.774 , 

KeyboardInterrupt: 

In [31]:
#to send files to storage bucket

!gsutil cp checkpoint_full2.tar gs://marine-clarity-307511-aiplatform  

Copying file://checkpoint_full1.tar [Content-Type=application/x-tar]...
/ [1 files][ 60.2 MiB/ 60.2 MiB]                                                
Operation completed over 1 objects/60.2 MiB.                                     
